# TP3 - Arbre et Random Forest : Prédiction du churn

Issue d'un [dataset](https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset) disponible sur Kaggle.

Le dataset pour cette séance correspond à des clients qui ont quitté ou non une banque. On souhaite savoir si le client va quitter la banque. Nous allons utiliser des arbres de décisions et des random forest pour répondre à ce problème.

## Contrôle de la qualité de donnée

Commençons par importer les données et les observer.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="whitegrid")

df = pd.read_csv("ChurnPrediction.csv")
df.head(10)

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,15574012,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,15592531,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,15656148,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,15792365,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,15592389,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


La colonne *customer_id* est unique et ne sert pas dans la prédiction. Notons que nous avons à la fois des données numérique et catégorielle.

**Consigne** : Supprimer la colonne *customer_id*

**Consigne** : En utilisant la méthode [*describe*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html), identifier s'il y a des valeurs qui paraissent aberrante dans les données numériques.

**Consigne** : Que peut-on dire des colonnes *tenure*, *products_number*, *credit_card* et *active_member* ?

**Consigne** : Calculer la proportion de déséquilibre.

**Consigne** : En utilisant la fonction *agregate_column*, explorer les champs catégoriels.

In [5]:
def agregate_column(column):
    grouped = df.groupby(by=column, as_index=False).agg('mean')
    return grouped[[column, "churn"]]

## Préparation des données

Maintenant que l'on a *un peu* observé les données, il nous reste à les préparer pour l'entraînement.

**Consigne** : Séparer le dataset en *X* et *y*

Puisque *X* est composé de donnée numérique comme catégorielle et que l'implémentation scikit-learn ne peut pas prendre en compte les données catégorielles, il faut les convertir.

**Consigne** : en utilisant la méthode [*get_dummies*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html), convertir avec la méthode One-Hot-Encoding les données catégorielles en données numérique. On aura prit soin de capitaliser sur les observations précédentes.

## Modélisation : Arbre

On souhaite prédire le churn a partir des données que l'on vient de préparer à l'aide d'un arbre de décision. Nous allons réaliser une validation croisée pour avoir une meilleure vision des performances de l'algorithme.
Cependant, le dataset est déséquilibré, donc nous ne pouvons pas réaliser une validation croisée sans prendre en compte ce déséquilibre.

**Consigne** : Avant de régler ce problème, Construire une fonction *performance* qui prend en paramètre un vecteur *vector* et qui affiche la moyenne et l'écart-type au format suivant : *moyenne (+/- ecart-type)*. On veillera à transformer le vecteur au format *numpy* avant les traitements.

In [9]:
def cross_validation_performance(vector):
    ...

**Consigne** : Compléter le code suivant. Il utilise la méthode [*StratifiedKFold*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) pour entraîner un [arbre de décision](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn-tree-decisiontreeclassifier). Puis afficher les performances avec la fonction *cross_validation_performance*.

On ne souhaite plus avoir ce bloc de code systématique, nous allons donc en faire une fonction. Pour pouvoir tester plusieurs paramétrage de l'arbre, on doit être capable de lui fournir des paramètres. Voici un exemple de l'utilisation :

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

cv = 5
folds = StratifiedKFold(n_splits=cv).split(X, y)
performances = []

for (train_index, test_index) in folds:
    X_train, X_test = ..., ...
    y_train, y_test = ..., ...
    
    model_trained = DecisionTreeClassifier().fit(..., ...)
    y_pred = model_trained.predict(...)
    ...

cross_validation_performance(performances)

In [ ]:
parameters = {
    "criterion": "gini",
    "max_depth": 8,
    "min_samples_leaf": 20
}

model = DecisionTreeClassifier(**parameters)

**Consigne** : En exploitant ce fonctionnement, construire une fonction *stratified_cross_validation* qui prends en paramètre :
* *X*: le dataset des features
* *y*: le vecteur réponse
* *model*: le modèle que l'on veut tester, au format scikit-learn
* *parameters*: le dictionnaire de paramètres à transmettre à *model*
* *metric*: la métrique avec laquelle on mesure les performances de *model*, au format scikit-learn
* *cv*: le nombre de pli de la validation croisée

Elle devra renvoyer les performances sur chacun des plis.

In [ ]:
def stratified_cross_validation(X, y, model, parameters, metric=f1_score, cv=3):
    ...

## Impact de la profondeur

On souhaiterai mesurer l'importance de la profondeur d'un arbre pour ce problème.

**Consigne** : A l'aide de la fonction précédente, répondre à la problématique avec un affichage.

Cette performance correspond en réalité au seuil 0.5. On souhaiterai être capable de trouver un seuil qui maximise le f1-score. 

## Trouver le seuil qui maximise une métrique

Pour le faire, nous allons avoir besoin de trois bases :
* Une base d'entraînement (*X_train*, *y_train*) : **entraîner** le modèle
* Une base de validation (*X_valid*, *y_valid*) : **trouver** le meilleur seuil
* Une base de test (*X_test*, *y_test*) : **tester** la performance sur des données non vues

**Consigne** : Générer les trois bases à l'aide la fonction [*train_test_split*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split), en prenant soin de conserver le même déséquilibre sur les trois bases.

**Consigne** : Entraîner un arbre puis prédire les probabilités d'être de la classe d'intérêt pour le dataset de validation. Les stocker dans une variable *y_proba*.

**Consigne** : Construire une fonction *find_best_treshold* qui prends en paramètre :
* *y_true* : vecteur des classes attendues
* *y_proba* : vecteur de probabilité estimé des classes
* *metric* : métrique à optimiser, au format scikit-learn
Elle revoit la meilleure performance et le meilleur seuil pour la métrique sélectionnée

In [16]:
def find_best_threshold(y_true, y_proba, metric):
    ...

**Consigne** : Utiliser la fonction *find_best_threshold* sur le jeu de validation, et comparer avec la performance obtenue sur le jeu de test.

**Consigne** : Reprendre la fonction *stratified_cross_validation* et la modifier pour afficher la meilleure performance que l'on puisse obtenir, avec en plus la valeur du seuil.

In [18]:
def stratified_cross_validation(X, y, model, parameters, metric=f1_score, cv=3):
    ...

## Impact de la profondeur : le retour

Maintenant que l'on sait obtenir la meilleur version de chaque algorithme, on souhaite mesurer un peu mieux l'impact de la profondeur.

**Consigne** : A l'aide de la fonction précédente, répondre à la problématique avec un affichage.

On souhaiterai avoir une représentation visuelle de cet affichage. Pour ce faire, on définit la fonction suivante.

In [20]:
def plot_performance(parameters, performances, color=None, label=None,confidence=3):
    if color is None: color=sns.color_palette()[0]
    if label is None: label=""
        
    mean = [performance.mean() for performance in performances]
    deviation = [performance.std() for performance in performances]
    
    mean, deviation = np.array(mean), np.array(deviation)
    
    plt.fill_between(parameters, mean - confidence*deviation, mean + confidence*deviation, alpha=0.15, color=color)
    plt.plot(parameters, mean, 'o-', color=color, label=label)

**Consigne** : en reprenant la question précédente (en adaptant), et en utilisant la fonction *plot_performance*, montrer visuellement l'impact de la profondeur sur la performance.

## Et la Random Forest ?

On s'intéresse maintenant à la Random Forest. On souhaite mesurer la même chose que pour l'arbre.

**Consigne** : reproduire la même étude, mais avec une Random Forest de 50 arbres.

**Consigne** : Afficher sur le même graphique, avec une légende, les performances pour un arbre et pour une Random Forest.

## Et maintenant ?

Il existe d'autres hyperparamètres important dans ces modèles. Reproduire les études, et comparer les performances entre arbres et Random Forest voire Extra-Trees.